In [1]:
from IPython.display import HTML
from IPython.display import display

# Taken from https://stackoverflow.com/questions/31517194/how-to-hide-one-specific-cell-input-or-output-in-ipython-notebook
tag = HTML('''<script>
code_show=false; 
function code_toggle() {
    if (code_show){
        $('div.cell.code_cell.rendered.selected div.input').hide();
    } else {
        $('div.cell.code_cell.rendered.selected div.input').show();
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Pour montrer la solution, cliquer <a href="javascript:code_toggle()">ici</a>.''')
display(tag)

# Le Machine Learning, c'est pour tout le monde

## Le Machine Learning, kézako ?

Le Machine Learning, ou apprentissage automatique en français, est une façon de programmer les ordinateurs de façon à ce qu'ils exécutent une tâche souhaité sans avoir programmé explicitement les instructions pour cette tâche.

En programmation classique, on a des données en entrée (input), une suite d'instruction qui vont s'appliquer sur ces données, et un résultat en sortie (output). 

En Machine Learning, on a des données en entrée, qui vont être fournies à un programme qui va "apprendre" un modèle de façon à ce que le résultat en sortie corresponde à ce que l'on souhaite. 
Dans un second temps, on peut donner de nouvelles données au modèle qui va donc produire des résultats nouveaux. Si ces résultats correspondent à ce que l'on attendait, alors on est content et on dit que le modèle a appris.

![Un schéma programmation classique vs ML](MLvsProg.png)

L'apprentissage, ça n'est pas une étape magique, c'est juste la solution d'un problème mathématique d'optimisation. En gros, on cherche à minimiser l'erreur dans un espace abstrait qui contient plein de modèles de Machine Learning. 

Ca peut paraître un peu compliqué, mais il y a plein de mathématiciens qui ont réfléchi à la question depuis longtemps, et on a des algorithmes efficaces pour ça. 

![Une illustration de la descente de gradient](3d-gradient_descent.png)

*Une illustration de la technique d'optimisation de descente de gradient, qui consiste à suivre la plus forte pente pour trouver le minimum (un peu comme une bille lâchée du haut d'une montagne qui va aller s'arrêter au fond de la vallée).*

**Pourquoi appelle-t-on ça un modèle de Machine Learning, et non pas un programme comme tout le monde ?**
D'abord, parce qu'on est un peu snob, et ensuite, parce que les modèles programmés peuvent aussi souvent s'exprimer en termes mathématiques, et on peut faire des preuves dessus, ect. 

Un modèle de Machine Learning, c'est une sous-espèce de programme informatique, qui est codé de façon un peu différente, c'est tout.

## Si les machines peuvent apprendre, pourquoi s'embêter à programmer ?

Donc pour faire du Machine Learning, il faut un problème, une tâche à résoudre, et des exemples de comment on veut que la tâche soit réalisée. 

Ce besoin de données (les exemples) explique pourquoi on ne se sert pas du Machine Learning partout. Il y a des tas de tâches pour lesquelles il est plus simple d'expliquer à l'ordinateur comment faire étape par étape, de lui donner les instructions, plutôt que de trouver tout un tas d'exemples du comportement qu'on veut apprendre. Le Machine Learning ne remplace donc absoulement pas la programmation ! 

Par contre, il y a des tâches pour lesquelles c'est plus compliqué d'expliquer comment on veut que ça soit fait. Par exemple, décider de si une chanson est triste ou au contraire joyeuse. On peut bien trouver des idées de pourquoi telle chanson est mélancolique ou telle autre nous met en pleine forme, les chansons triste sont peut-être plus lentes, les chansons joyeuses ont souvent de la batterie, mais c'est compliqué d'expliquer à un ordinateur comment faire la différence. Par contre, on peut facilement classer les chansons dans deux catégories, triste et joyeuse, et donner cette classification ainsi que le spectre audio des chansons à un ordinateur, et lui demander d'apprendre de lui-même à reconnaître les chansons tristes ou joyeuses !


## Mon premier modèle de Machine Learning

Pour notre premier modèle, on ne va pas apprendre à distinguer les chansons tristes des autres, on va classer quelque chose de plus simple. 

On a un ensemble de données (un dataset en anglais) qui, pour chaque configuration d'une petite expérience de physique, nous dit si la balance penche vers la droite ou vers la gauche. 

![L'expérience de la balance](balance.png)

On a quatre variables pour chaque expérience: la longueur du bras gauche (`dist_gauche`), la longueur du bras droit (`dist_droit`), la masse sur le plateau gauche (`masse_gauche`) et la masse sur le plateau droit (`masse_droit`). On veut que l'ordinateur apprenne le résultat de l'expérience (la balance penche à gauche ou à droite) à partir de ces quatre variables. 

Evidemment, dans ce cas très simple, un peu de physique pourrait nous permettre de trouver la solution et de programmer explicitement les instructions. Mais supposant qu'on a oublié nos cours de physique. Que peut-on faire ?

On commence par charger les données, les exemples d'expérience pour lesquels on a la réponse. Ces données sont dans un fichier csv, un fichier texte où les valeurs sont séparées par des virgules (*comma-separated values* en anglais). 

On utilise le module csv de la librairie standart pour stocker nos données dans une grande liste, où chaque élément représentera une expérience. 

Le code pour charger les données est donné à titre indicatif, pas la peine de s'en inquiéter pour l'instant !

In [2]:
import csv

train_dataset = []
with open('apprentissage.csv') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        train_dataset.append({'label': row[0], 'variables': list(map(int, row[1:]))})    

Une expérience (un point de donnée) est caractérisée par les valeurs des quatre variables dont on a parlé ci-dessus (`masse_gauche`, `dist_gauche`, `masse_droite`, `dist_droite`), et par son **label** (le résultat de l'expérience, est-ce que la balance penche à droite (**R**) ou à gauche (**L**). 

On peut par exemple regarder le premier point de données du dataset.

In [3]:
train_dataset[0]

{'label': 'R', 'variables': [5, 4, 5, 5]}

On voit que les variables `masse_gauche`, `dist_gauche`, `masse_droite`, `dist_droite` ont pour valeur respectivement 5, 4, 5 et 5, et que le résultat de l'expérience est que la balance penche à droite (label **R**). 

Toutes les valeurs des variables ont été normalisées entre 1 et 5, c'est une pratique courante en Machine Learning pour simplifier l'apprentissage. 

Maintenant que l'on a des données, comment va-t-on faire pour que l'ordinateur apprenne ? 

On va utiliser l'algorithme des plus proches voisins.

Cet algorithme fonctionne sur le principe suivant: on va regarder quels sont les points qui sont proches du point qu'on veut évaluer en utilisant les variables qu'on a collectées. Concrètement, on cherche dans notre dataset d'apprentissage les expériences qui sont les plus similaires à l'expérience qu'on veut évaluer. Et on se dit que les choses vont se passer à peu près pareil pour notre expérience inconnue, et donc on va renvoyer le même label (on prédit la même classe). 

![k-NN en 2D](knn.png)

Sur le schéma ci-dessus, le point vert est entouré de points rouges, on va donc lui assigner le label rouge. 

Pour le point violet, c'est un peu plus compliqué: si on prend le point le plus proche, il serait bleu; mais si on fait la moyenne sur les 3 points les plus proches, il serait rouge. 

En pratique, on fait souvent la moyenne sur quelques points (3 ou 5), afin de lisser les cas limites (comme celui du point violet). 

# Notre modèle de Machine Learning va être ici composé de 3 choses : 
* le dataset d'apprentissage qu'on va utiliser pour trouver les voisins
* la valeur *k* du nombre de voisins qu'on va considérer pour faire nos évaluations
* la fonction qu'on va utiliser pour calculer la distance

Commençons par définir cette fonction de distance. 

On va utiliser une distance dite *de Manhattan*: 
$$ d(a, b) = |a_1 - b_1| + |a_2 - b_2| + ...$$
qui calcule la différence sur chaque variable, puis somme les valeurs absolues de ces différences. 

On va donc parcourir toutes les variables des points de données (les expériences) *a* et *b* et ajouter la valeur absolue de la différence. 

La fonction `zip(a, b)` va renvoyer *a_1, b_1*, puis *a_2, b_2*, ect...

C'est à vous de jouer, complétez le code ci-dessous ! 

(Un dernier indice, la fonction `math.fabs` vous donnera la valeur absolue.)

In [3]:
import math
def distance(a, b):
    d = 0
    for i, j in zip(a, b):
        d += ... # à compléter 
    return d

In [4]:
import math
def distance(a, b):
    d = 0
    for i, j in zip(a, b):
        d += math.fabs(i - j)
    return d
display(tag)

Pour tester si notre fonction fonctionne correctement, on va calculer la distance entre les deux premiers points du dataset. Vous devrier obtenir 11.

In [ ]:
distance(train_dataset[0]['variables'], train_dataset[1]['variables'])

Pour le nombre de voisins que l'on va considérer, on va prendre *k = 3*. 

In [ ]:
k = 3

C'est parti, on peut à présent se lancer dans le coeur de l'algorithme. 

Quand on reçoit un point à évaluer, on veut connaître les labels des *k* plus proches voisins. Pour cela, on va calculer la distance du nouveau point à tous les points du dataset d'apprentissage, et stocker à chaque fois la valeur et le label.

(Indice: on a déjà écrit une fonction pour calculer la distance; on veut prendre uniquement la partie 'variables' du `train_point` pour calculer la distance.)

In [6]:
def calculer_distances(dataset, eval_point):
    distances = []
    for train_point in dataset:
        distances.append({'label': train_point['label'], 
                          'distance': ... # à compléter
                         })
    return distances

In [7]:
def calculer_distances(dataset, eval_point):
    distances = []
    for train_point in dataset:
        distances.append({'label': train_point['label'], 
                          'distance': distance(eval_point, train_point['variables']) 
                         })
    return distances
display(tag)        

Maintenant qu'on a toutes les distances, on peut trouver les *k* points les plus proches, en triant par distance. Python nous permet de faire ça facilement, mais les détails sont un peu techniques.

In [ ]:
def trier_distances(distances):
    return sorted(distances, key=lambda point: point['distance'])

On peut prendre les trois premiers éléments de la liste avec `maListe[:3]`, et compter le nombre de **R** et de **L** dans les labels.

In [ ]:
def choisir_label(distances_triees, k):
    plus_proches_voisins = distances_triees[:k]
    count_R = 0
    count_L = 0
    for voisin in plus_proches_voisins:
        if voisin['label'] == 'R':
            count_R += 1
        if voisin['label'] == 'L': 
            ... # à compléter
    if count_R > count_L:
        return 'R'
    else:
        return 'L'
    

In [8]:
def choisir_label(distances_triees, k):
    plus_proches_voisins = distances_triees[:k]
    count_R = 0
    count_L = 0
    for voisin in plus_proches_voisins:
        if voisin['label'] == 'R':
            count_R += 1
        if voisin['label'] == 'L': 
            # à compléter
            count_L += 1
    if count_R > count_L:
        return 'R'
    else:
        return 'L'
display(tag)

A présent on combine toutes nos fonctions pour avoir l'algorithme final.

In [ ]:
def algorithme_des_plus_proches_voisins(dataset, eval_point, k):
    distances = calculer_distances(dataset, eval_point)
    distances_triees = trier_distances(distances)
    label = choisir_label(distances_triees, k)
    return label

Maintenant, on va charger de nouvelles données pour pouvoir évaluer notre algorithme. 

In [ ]:
import csv

eval_dataset = []
with open('evaluation.csv') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in csvreader:
        eval_dataset.append({'true_label': row[0], 'variables': list(map(int, row[1:]))})    

On peut regarder ce que ça donne pour notre premier point d'évaluation.

In [ ]:
algorithme_des_plus_proches_voisins(train_dataset, eval_dataset[0]['variables'], k)

On prédit que la balance va pencher à gauche ! 

Qu'en est-il en réalité ?

In [ ]:
eval_dataset[0]['true_label']

C'est bien ce qui a été observé ! Notre algorithme a appris, sans avoir besoin de lui expliquer les règles de la physique ! 

Voyons combien d'erreurs notre algorithme fait sur la totalité des points d'évaluation.

In [ ]:
erreurs = 0
for point in eval_dataset:
    if algorithme_des_plus_proches_voisins(train_dataset, point['variables'], k) != point['true_label']:
        erreurs += 1
print("Nombre d'erreurs %d sur %d exemples" % (erreurs, len(eval_dataset)))

Notre algorithme a fait quelques erreurs, mais il a correctement prédit la majorité des situations. 

Félicitations, vous avez programmé votre premier modèle de Machine Learning ! 